# Projecting the National Elevation Dataset to North America Albers Equal Area Conic using GDAL

## Required:
 - Unprojected NED Cloud Optimized GeoTIFFS (COGs)
     - A globebd list of COGs that covers the whole country (1/3rd arc, 1/9th arc, 1m)
 - GDAL Warp command line function
 
## Workflow

 1. Set paths for input rasters, output porjected rasters and the .cmd file that will do all the hard work for us
 2.  Glob for a list of all unprojected COGs
 3.  Define the function that will write the .cmd file
     - Easiest to write it to the same folder that the gdal tool is located in
 4.  Write the .cmd file
 5. Open cmd prompt, cd to the location of the folder containing both the gdal tool and the .cmd file, run the .cmd file
     - Better than just opening the .cmd so that you can see the cmd prompt log and identify errors when it is finished/crashes
 6. Check to see if any COGs were missed

In [1]:
# Import the necessary modules
import os
from glob import glob

### Workflow step 1.
#### Set paths for input rasters, output porjected rasters and the .cmd file that will do all the hard work for us

In [2]:
# Set paths
inputs_tif_dir = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED_COGTIFFS'
outputs_tif_dir = r'T:\CCSI\TECH\FEMA\2017_NFIP_RiskRating_Milliman\GIS\Harddrive\NED\NED_COGTIFFS_AEA'
cmd_file = r'C:\Users\abrazeau\Desktop\test_warp_processing\RunBatch.cmd'

### Workflow step 2.
#### Glob for a list of all unprojected COGs

In [3]:
# File types, globbed list of COGs
file_type = '.tif'
tifs = glob(os.path.join(inputs_tif_dir, '*{}'.format(file_type)))

# SANITY CHECK
print('{} Cloud Optimized GeoTIFFS found in input directory'.format(len(tifs)))

13199 Cloud Optimized GeoTIFFS found in input directory


### Workflow step 3.
#### Define the function that will write the .cmd file

In [4]:
# MAIN FUNCTION
def WriteCMD(cmd_file, raster_paths, file_type, gdal_function, new_file_dir):
    '''This function opens a .cmd file in write mode, goes through a list of rasters and writes a command to use the gdal warp
    tool on each raster. This tool projects a raster to a specified coordinate system, then the function places the projected COG
    in 'new_file_dir'. The new GeoTIFFs are saved using their same file name from before projection.'''
    quotes = '"'
    with open(cmd_file, 'w') as f:
        for tif in raster_paths:
            cmd = '{} '.format(gdal_function)
            crs_options = ' -t_srs "+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' #NA Albers Eq. Area PROJ4 crs
            gdal_options = ' -co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW' # Maintain Cloud Optimized GeoTIFF properties
            if '.tif' in tif:
                raw_file_name = os.path.basename(tif).strip(file_type)
            else:
                raw_file_name = tif.split('\\')[-2]
            new_file_name = os.path.join(new_file_dir, raw_file_name + '.tif')
            terminal_input =cmd + crs_options + quotes+ ' ' + quotes + tif +quotes+ ' '+ quotes + new_file_name + quotes + gdal_options + '\n'
            f.write(terminal_input)
    return None

### Workflow step 4.
#### Write the .cmd file

In [5]:
# GDAL COMMAND
gdalwarp = 'gdalwarp'

WriteCMD(cmd_file, tifs, file_type, gdalwarp, outputs_tif_dir)

print ('The .cmd file to project all NED rasters can be found at {}'.format(cmd_file))

The .cmd file to project all NED rasters can be found at C:\Users\abrazeau\Desktop\test_warp_processing\RunBatch.cmd


### Workflow step 5.
#### Open cmd prompt, cd to the location of the folder containing both the gdal tool and the .cmd file, run the .cmd file

### Workflow step 6.
#### Check to see if any COGs were missed

In [6]:
itifs = glob(os.path.join(inputs_tif_dir, '*{}'.format(file_type)))
otifs = glob(os.path.join(outputs_tif_dir, '*{}'.format(file_type)))

print ('There are ', len(itifs) - len(otifs), ' COGs missing...')

There are  0  COGs missing...


#### Run this cell to find which COGs are missing

In [ ]:
# If the cell above prints any number but 0, run this cell to find which files are missing
tif_files = []
for itif in itifs:
    tif_files.append(tif.split('NED_COGTIFFS\\')[1])

otif_files = []
for otif in otifs:
    otif_files.append(otif.split('AEA\\')[1])
    
otifs_items = set(otif_files)
missing = [item for item in tif_files if item not in otifs_items]
print( 'The following COGs were missed: ', missing)

## References

##### CRS projection = North America Albers Equal Area Conic, ESRI:102008, info at https://epsg.io/102008
##### Adding 'Start ' to the beginning of each line in the .cmd file begins all wraps at once.. would crash the machine
##### Cloud Optimized GeoTIFF information at: https://trac.osgeo.org/gdal/wiki/CloudOptimizedGeoTIFF#HowtoreaditwithGDAL
##### gdalwarp info at: http://www.gdal.org/gdalwarp.html